<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Выявление-и-устранение-аномалий" data-toc-modified-id="Выявление-и-устранение-аномалий-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Выявление и устранение аномалий</a></span></li><li><span><a href="#Выявление-и-заполнение-пропусков" data-toc-modified-id="Выявление-и-заполнение-пропусков-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Выявление и заполнение пропусков</a></span></li><li><span><a href="#Работа-с-переменными" data-toc-modified-id="Работа-с-переменными-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Работа с переменными</a></span></li><li><span><a href="#Преобразование-категориальных-переменных" data-toc-modified-id="Преобразование-категориальных-переменных-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Преобразование категориальных переменных</a></span></li><li><span><a href="#Стандартизация" data-toc-modified-id="Стандартизация-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Стандартизация</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#kNN" data-toc-modified-id="kNN-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>kNN</a></span></li><li><span><a href="#Линейная-регрессия" data-toc-modified-id="Линейная-регрессия-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Линейная регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Проверка-на-тесте" data-toc-modified-id="Проверка-на-тесте-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка на тесте</a></span></li><li><span><a href="#Итоговые-выводы" data-toc-modified-id="Итоговые-выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Итоговые выводы</a></span></li></ul></div>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Определение стоимости автомобилей

## Описание проекта

**Заказчик** - сервис по продаже автомобилей с пробегом «Не бит, не крашен». Они занимаются разработкой приложение с целью привлечения новых клиентов. Приложение позволит пользователям узнавать рыночную стоимость своего автомобиля.

**Исходные данные** - информация о технических характеристиках, комплектации и ценах других автомобилей.

Заказчик выделил три основных критерия:
* качество предсказания - значение метрики RMSE должно быть меньше 2500;
* время обучения модели;
* время предсказания модели.

Исходный набор данных состоит из следующих признаков:
* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* NotRepaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя
Целевой признак
* Price — цена (евро)

## Описание данных

Начнем с загрузки исходных данных.

In [4]:
try:
    data = pd.read_csv('/datasets/autos.csv')
except:
    data = pd.read_csv('autos.csv')

Выведем основную информацию по исходным данным.

In [5]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Исходные данные насчитывают почти 355 тыс. наблюдений по 16 основным характеристикам. Тип характеристик разделен примерно пополам - `object` и `int`. В данных присутствуют наблюдения с пропущенными значениями. Перед этапом моделирования необходимо будет выявить и обработать пропуски с целью сохранения наибольшего объема данных для обучения. 

Большинство переменных с типом `object` будет необходимо перекодировать в классические категориальные переменные, чтобы с ними могли работать модели. 

Также часть переменных будут удалена вследствие их неинформативности. К таким переменным относятся `DateCrawled`, `DateCreated`, `PostalCode` и `LastSeen`. Однако перед удалением необходимо будет создать переменную, отвечающую за срок использования автомобиля до продажи. Переменную, отвечающую за тип модели машины также было решено удалить. 

Отдельно отметим необходимость проанализировать данные на потенциальные аномалии - пускай их не было видно при первичном анализе, не стоит исключать возможность их наличия.

## Предобработка данных

Перейдем к этапу предобработки. На данном этапе нам будет необходимо выявить и устранить пропуски, проверить данные на наличие аномалий и в случае чего устранить их, преобразовать часть переменных и, наконец, удалить неинформативные признаки.

### Выявление и устранение аномалий

Перейдем к следующему этапу предобработки - анализом данных на аномалии. Пускай первичный анализ не подтвердил их наличие, нами было принято решения провести более тщательный анализ - поскольку судить о почти 355 тыс. наблюдений по 5 первым строкам не совсем корректно.

Начнем с анализа переменных категориального типа. Выведем уникальные значения каждой из них и проверим, что нигде не присутствуют странные или нетипичные категории.

In [6]:
for name in ['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'NotRepaired']:
    display(data[name].unique())

array([nan, 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

array(['manual', 'auto', nan], dtype=object)

array(['petrol', 'gasoline', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'opel', 'seat',
       'citroen', 'honda', 'fiat', 'mini', 'smart', 'hyundai',
       'sonstige_autos', 'alfa_romeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'toyota', 'chevrolet', 'dacia',
       'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar', 'daewoo',
       'porsche', 'rover', 'land_rover', 'lada'], dtype=object)

array([nan, 'yes', 'no'], dtype=object)

Как можно заметить, существенные аномалии в категориях отсутствуют.

Перейдем к рассмотрению числовых характеристик.

Для проверки данных на аномалии выведем дескриптивную статистику численных характеристик.

In [7]:
data[['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']].describe()

,RegistrationYear,Power,Kilometer,RegistrationMonth
count,354369.000000,354369.000000,354369.000000,354369.000000
mean,2004.234448,110.094337,128211.172535,5.714645
std,90.227958,189.850405,37905.341530,3.726421
min,1000.000000,0.000000,5000.000000,0.000000
25%,1999.000000,69.000000,125000.000000,3.000000
50%,2003.000000,105.000000,150000.000000,6.000000
75%,2008.000000,143.000000,150000.000000,9.000000
max,9999.000000,20000.000000,150000.000000,12.000000


Итак, что можно заметить по полученной информации.

Прежде всего, обнаружены аномальные значения у переменной `RegistrationYear` - недопустимые значения минимума и максимума.

Аномальные значения присутствуют также и у переменной, отвечающей за мощность - также недопустимый максимум и минимум (на текущий момент мощность двигателя самого быстрого автомобиля лишь слегка превышает 2 000 л.с.). 

Относительно остальных переменных можно сделать вывод о том, что аномалий в них не обнаружено. Однако отдельно отметим тот факт, что `RegistrationMonth` принимает нулевые значения - мы склонны полагать, что это своего рода заглушка для тех случаев, когда месяц регистрации не был указан.

Перейдем к обработке аномалий. Начнем с переменной `RegistrationYear`. Для начала рассмотрим те наблюдения, у которых регистрация была совершена неестественно рано.

In [8]:
data[(data['RegistrationYear'] < 1950)].shape

(246, 16)

Наблюдений достаточно много для индивидуальной обработки, к тому же довольно сложно определить, какой год мог бы быть уместным для замены. На основании этого нами было принято решение удалить такие наблюдения.

In [9]:
data = data.drop(labels = data[(data['RegistrationYear'] < 1950)].index)

In [10]:
data[(data['RegistrationYear'] < 1940)]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen


Как можно заметить, удаление прошло успешно.

Перейдем к следующей группе дат. Поскольку дата скачивания максимально датируется 2016 годом, рассмотрим все те наблюдения, которые имеют более поздний год.

In [11]:
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], format = '%Y-%m-%d')
data['DateCrawled'].dt.year.max()
# вывод максимального года скачивания анкеты

2016

In [12]:
data[(data['RegistrationYear'] > 2016)].head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
22,2016-03-23 14:52:51,2900,NaN,2018,manual,90,meriva,150000,5,petrol,opel,no,2016-03-23 00:00:00,0,49716,2016-03-31 01:16:33
26,2016-03-10 19:38:18,5555,NaN,2017,manual,125,c4,125000,4,NaN,citroen,no,2016-03-10 00:00:00,0,31139,2016-03-16 09:16:46
48,2016-03-25 14:40:12,7750,NaN,2017,manual,80,golf,100000,1,petrol,volkswagen,NaN,2016-03-25 00:00:00,0,48499,2016-03-31 21:47:44
51,2016-03-07 18:57:08,2000,NaN,2017,manual,90,punto,150000,11,gasoline,fiat,yes,2016-03-07 00:00:00,0,66115,2016-03-07 18:57:08
57,2016-03-10 20:53:19,2399,NaN,2018,manual,64,other,125000,3,NaN,seat,no,2016-03-10 00:00:00,0,33397,2016-03-25 10:17:37


Предположим, что все те наблюдения, у которых год регистрации указан до 2020, имеют опечатку - на самом деле год должен быть на 10 лет меньше. Исходя из этого предположения, обработаем часть аномалий.

In [13]:
data.loc[(data['RegistrationYear']>2016)&(data['RegistrationYear']<2020), 
         'RegistrationYear'] = data['RegistrationYear'] - 10

Рассмотрим оставшиеся аномалии.

In [14]:
data[(data['RegistrationYear'] > 2019)].head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
12946,2016-03-29 18:39:40,49,NaN,5000,NaN,0,golf,5000,12,NaN,volkswagen,NaN,2016-03-29 00:00:00,0,74523,2016-04-06 04:16:14
15147,2016-03-14 00:52:02,0,NaN,9999,NaN,0,NaN,10000,0,NaN,sonstige_autos,NaN,2016-03-13 00:00:00,0,32689,2016-03-21 23:46:46
15870,2016-04-02 11:55:48,1700,NaN,3200,NaN,0,NaN,5000,0,NaN,sonstige_autos,NaN,2016-04-02 00:00:00,0,33649,2016-04-06 09:46:13
17271,2016-03-23 16:43:29,700,NaN,9999,NaN,0,other,10000,0,NaN,opel,NaN,2016-03-23 00:00:00,0,21769,2016-04-05 20:16:15
17346,2016-03-06 16:06:20,6500,NaN,8888,NaN,0,NaN,10000,0,NaN,sonstige_autos,NaN,2016-03-06 00:00:00,0,55262,2016-03-30 20:46:55


Слишком много наблюдений для индивидуального заполнения, к тому же слишком много частных случаев для общего заполнения. Нами принято решения удалить данные наблюдения.

In [16]:
data = data.drop(labels = data[(data['RegistrationYear'] > 2019)].index)

Перейдем к обработке следующей переменной `Power`. Официально существующий максимум мощности - 2028 л.с. Мы склонны полагать, что все значения мощности превышающие 1000 л.с. - опечатка или ошибка. 

Предположим, что к ним было случайно добавлено лишнее число - тогда возьмем остаток от деления на 10. Отдельно рассмотрим мощности от 4000 л.с. и выше. У них, вероятнее всего, было два лишних символа - будет браться остаток от деления на 100.

In [17]:
data.loc[data['Power'] >=4000, 'Power'] = data['Power'] // 100
data.loc[data['Power'] >=1000, 'Power'] = data['Power'] // 10

Также имеются аномалии относительно слишком маленьких мощностей. За порог возьмем мощность в 10 л.с. Попробуем заполнить аномальные мощности медианными значениями относительно бренда автомобиля.

In [18]:
brand_power_pivot = data.pivot_table(index='Brand', 
                                     values='Power', aggfunc = 'median').reset_index()

brand_power_pivot.head()

,Brand,Power
0,alfa_romeo,140.0
1,audi,143.0
2,bmw,150.0
3,chevrolet,110.0
4,chrysler,143.0


Как можно заметить, слишком странных или аномальных медианных значений нет - можем использовать для заполнения. 

In [19]:
def power_anomaly(row):
    power = row['Power']
    brand = row['Brand']
    if power <= 10:
        for i in range(len(brand_power_pivot)):
            if brand_power_pivot['Brand'][i] == brand:
                return brand_power_pivot['Power'][i]
    else: return power

In [20]:
data['Power'] = data.apply(power_anomaly, axis = 1)

In [21]:
data[data['Power'] <= 10]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen


Итак, нам удалось устранить выявленные аномалии. Переходим к следующему этапу работы. 

### Выявление и заполнение пропусков

Начнем с работы над пропусками. Выведем долю наблюдений с пропущенными значениями в каждом из столбцов.

In [22]:
for name in data.columns:
    print(f'Доля пропусков в столбце {name}:', '%.3f' % data[name].isna().mean())

Доля пропусков в столбце DateCrawled: 0.000
Доля пропусков в столбце Price: 0.000
Доля пропусков в столбце VehicleType: 0.105
Доля пропусков в столбце RegistrationYear: 0.000
Доля пропусков в столбце Gearbox: 0.055
Доля пропусков в столбце Power: 0.000
Доля пропусков в столбце Model: 0.055
Доля пропусков в столбце Kilometer: 0.000
Доля пропусков в столбце RegistrationMonth: 0.000
Доля пропусков в столбце FuelType: 0.092
Доля пропусков в столбце Brand: 0.000
Доля пропусков в столбце NotRepaired: 0.200
Доля пропусков в столбце DateCreated: 0.000
Доля пропусков в столбце NumberOfPictures: 0.000
Доля пропусков в столбце PostalCode: 0.000
Доля пропусков в столбце LastSeen: 0.000


Как можно заметить в 5 из исходных 16 характеристиках есть наблюдения с пропусками.

Для начала избавимся от тех наблюдений, где есть пропуски в максимальном числе характеристик. Мы склонны полагать, что это либо некачественные объявления, либо ложные. Потеря будет составлять только 0.6% от исходного объема данных, так что мы считаем данное решение допустимым.

In [23]:
data = data.dropna(subset=
                   ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired'], how='all')

Перейдем к заполнению оставшихся пропусков. 

Характеристики с пропусками - довольно специфичные особенности машин. Поскольку мы не имеем возможности восстановить информацию для каждого объявления в отдельности, заполним пропуски наиболее распространенным типом значений для каждой из характеристик.

Отдельно отметим, что мы оставим переменную, отвечающую за тип модели, с пропусками, поскольку позднее данная переменная будет удалена. 

In [24]:
for name in ['VehicleType', 'Gearbox', 'FuelType', 'NotRepaired']:
    mode = data[name].mode()
    data.loc[data[name].isna() == True, name] = mode.values[0]

Проверим, что в данных не осталось наблюдений с пропущенными значениями - кроме тех характеристик, которые мы решили не обрабатывать.

In [25]:
for name in data.columns:
    print(f'Доля пропусков в столбце {name}:', '%.3f' % data[name].isna().mean())

Доля пропусков в столбце DateCrawled: 0.000
Доля пропусков в столбце Price: 0.000
Доля пропусков в столбце VehicleType: 0.000
Доля пропусков в столбце RegistrationYear: 0.000
Доля пропусков в столбце Gearbox: 0.000
Доля пропусков в столбце Power: 0.000
Доля пропусков в столбце Model: 0.049
Доля пропусков в столбце Kilometer: 0.000
Доля пропусков в столбце RegistrationMonth: 0.000
Доля пропусков в столбце FuelType: 0.000
Доля пропусков в столбце Brand: 0.000
Доля пропусков в столбце NotRepaired: 0.000
Доля пропусков в столбце DateCreated: 0.000
Доля пропусков в столбце NumberOfPictures: 0.000
Доля пропусков в столбце PostalCode: 0.000
Доля пропусков в столбце LastSeen: 0.000


Действительно, нам удалось избавиться от наблюдений с пропусками.

### Работа с переменными

Перейдем к удалению неинформативных переменных.

In [26]:
for name in ['DateCrawled', 'DateCreated', 'Model', 'RegistrationMonth', 'PostalCode', 'LastSeen', 'NumberOfPictures']:
    del data[name]

In [27]:
data = data.reset_index(drop=True)

In [28]:
data.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Kilometer,FuelType,Brand,NotRepaired
0,480,sedan,1993,manual,90.0,150000,petrol,volkswagen,no
1,18300,coupe,2011,manual,190.0,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163.0,125000,gasoline,jeep,no
3,1500,small,2001,manual,75.0,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69.0,90000,gasoline,skoda,no


### Преобразование категориальных переменных

Наконец, перейдем к заключительному этапу первичной обработки данных - перекодирования категорий. 

Мы будем использовать два метода кодировки. One-Hot Encoding будет использоваться для `NotRepaired` и `Gearbox`, поскольку каждая из переменных принимает только по два значения. Остальные категории будут закодированы методом Ordinal Encoding.

Отдельно создадим набор данных, в котором будет использоваться One-Hot Encoding для всех переменных - на нем мы будем обучать линейную регрессию. 

Перед этим разделим данные на переменные и таргет.

In [29]:
X = data.copy().drop(columns='Price')
y = data[['Price']].copy()

In [30]:
X_ohe = pd.get_dummies(X, drop_first=True)

X_ohe.head()

,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
0,1993,90.0,150000,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2011,190.0,125000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2004,163.0,125000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2001,75.0,150000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2008,69.0,90000,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [31]:
X= X.join(pd.get_dummies(X[['NotRepaired', 'Gearbox']], 
                                      drop_first=True)).drop(['NotRepaired', 'Gearbox'], axis = 1).reset_index(drop=True)

In [32]:
X.head()

,VehicleType,RegistrationYear,Power,Kilometer,FuelType,Brand,NotRepaired_yes,Gearbox_manual
0,sedan,1993,90.0,150000,petrol,volkswagen,0,1
1,coupe,2011,190.0,125000,gasoline,audi,1,1
2,suv,2004,163.0,125000,gasoline,jeep,0,0
3,small,2001,75.0,150000,petrol,volkswagen,0,1
4,small,2008,69.0,90000,gasoline,skoda,0,1


Первый этап кодировки прошел успешно. Переходим ко второму.

In [33]:
encoder = OrdinalEncoder()

encoder.fit(X[['VehicleType', 'FuelType', 'Brand']])

OrdinalEncoder()

In [34]:
data_ordinal = pd.DataFrame(encoder.transform(X[['VehicleType', 'FuelType', 'Brand']]), 
                                columns = [['VehicleType', 'FuelType', 'Brand']]).reset_index(drop=True)
X[['VehicleType', 'FuelType', 'Brand']] = data_ordinal[['VehicleType', 'FuelType', 'Brand']]
    
X.head()

,VehicleType,RegistrationYear,Power,Kilometer,FuelType,Brand,NotRepaired_yes,Gearbox_manual
0,4.0,1993,90.0,150000,6.0,38.0,0,1
1,2.0,2011,190.0,125000,2.0,1.0,1,1
2,6.0,2004,163.0,125000,2.0,14.0,0,0
3,5.0,2001,75.0,150000,6.0,38.0,0,1
4,5.0,2008,69.0,90000,2.0,31.0,0,1


### Стандартизация

Перед переходом к завершающему этапу работы над данными проведем процедуру стандартизации для непрерывных переменных.

Однако перед этим разделим наши данные на тренировочную и тестовую выборки.

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                               test_size = 0.25,
                                               shuffle = True,
                                               random_state = 9)

X_train_ohe, X_test_ohe, y_train_ohe, y_test_ohe = train_test_split(X_ohe,y,
                                               test_size = 0.25,
                                               shuffle = True,
                                               random_state = 9)

Обучим скалер на тренировочных данных, после чего применим скалер ко всему набору данных.

In [36]:
columns_for_scaler = ['Power', 'Kilometer', 'RegistrationYear']
scaler = StandardScaler()

scaler.fit(X_train[columns_for_scaler])
X_train[columns_for_scaler] = scaler.transform(X_train[columns_for_scaler])
X_test[columns_for_scaler] = scaler.transform(X_test[columns_for_scaler])

scaler.fit(X_train_ohe[columns_for_scaler])
X_train[columns_for_scaler] = scaler.transform(X_train_ohe[columns_for_scaler])
X_test[columns_for_scaler] = scaler.transform(X_test_ohe[columns_for_scaler])

In [37]:
X_train.head()

,VehicleType,RegistrationYear,Power,Kilometer,FuelType,Brand,NotRepaired_yes,Gearbox_manual
181769,7.0,0.335437,-0.535041,0.573552,2.0,38.0,0,1
317372,4.0,0.782337,1.995588,0.573552,6.0,30.0,0,1
326679,4.0,-0.558365,-0.629468,0.573552,6.0,30.0,0,1
168606,1.0,0.782337,0.786929,-0.761568,6.0,1.0,0,1
41664,4.0,-0.409398,-0.818321,0.573552,6.0,27.0,0,1


Итак, нами была проведена работа по предобработке исходных данных. В ходе первичного анализа были обнаружены и впоследствии обработаны следующие моменты:
* пропущенные значения;

Присутствовали в 5 характеристиках. Часть была удалена, в остальных наблюдениях пропуски были заменены на наиболее распространенную категорию.

* аномальные значения;

Были обнаружены аномалии в двух непрерывных характеристиках: мощности двигателя и дате регистрации авто. Для каждого случая подбирался уникальный метод устранения.

* переменные недопустимого формата.

Часть переменных, отвечающих за некоторые категории, была записана недопустимым для моделирования способом. Мы провели перекодировку данных характеристик наиболее подходящим способом.

Дополнительно мы создали новую переменную и удалили часть исходных неинформативных параметров.

## Обучение моделей

Перейдем к следующему этапу работы - созданию модели для предсказания рыночной стоимости авто.

Всего нами будет рассмотрено 5 моделей: kNN, линейная регрессия, случайный лес, LightGBM и CatBoost. Для каждой из моделей мы попытаемся подобрать такие параметры, которые будут максимизировать показатель RMSE.

### kNN

Начнем с метода ближайших соседей. Подберем лучшие параметры.

In [38]:
knn = KNeighborsRegressor(n_jobs=-1)
params = { 'n_neighbors': [5, 15], 'p': [1, 2], 'weights': ['uniform', 'distance'] }
knn_cv = GridSearchCV(knn, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
knn_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(n_jobs=-1), n_jobs=-1,
             param_grid={'n_neighbors': [5, 15], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='neg_mean_squared_error')

Оценим время обучения лучшей модели и время предсказаний.

In [39]:
knn_cv.refit_time_

5.71871280670166

In [40]:
%%time
knn_cv.predict(X_train)

Wall time: 18.7 s


array([[ 3518.        ],
       [10699.        ],
       [  959.8       ],
       ...,
       [ 4999.        ],
       [ 2194.46666667],
       [ 2555.        ]])

Наконец перейдем к оценки качества модели.

In [41]:
print('RMSE метода k-ближайших соседей:', (-knn_cv.best_score_)**0.5)

RMSE метода k-ближайших соседей: 1910.4489957993765


Несмотря на то, что метод ближайших соседей показал неплохие результаты относительно времени обучения, а также качество модели существенно ниже необходимого порога. Мы оставляем данную модель для дальнейшего рассмотрения.

### Линейная регрессия

Теперь перейдем к модели линейной регрессии. Оценим скорость обучения.

In [42]:
%%time
lin_reg = LinearRegression().fit(X_train_ohe, y_train)

Wall time: 875 ms


Также проведем оценку скорости предсказывания.

In [43]:
%%time
lin_reg_pred = lin_reg.predict(X_train_ohe)

Wall time: 130 ms


Наконец, оценим качество полученной модели.

In [44]:
print('RMSE линейной регрессии:', mean_squared_error(y_train, lin_reg_pred,  squared=False))

RMSE линейной регрессии: 2938.596546647935


Несмотря на то, что линейная модель показала лучшее время обучения и предсказания по сравнению с методом ближайших соседей, ее качество не достигает минимального порога. На основании этого мы исключаем данную модель из дальнейшего рассмотрения.

### Случайный лес

Теперь перейдем к модели случайного леса. Как и прежде, начнем с подбора наиболее оптимальных параметров. 

In [45]:
forest = RandomForestRegressor(random_state=9, n_jobs=-1)
params_forest = {'max_depth': range (1, 7, 1), 'max_features': ['auto', 'sqrt', 'log2'], 
                 'n_estimators': range(90, 120, 10)}
forest_cv = GridSearchCV(forest, params_forest, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
forest_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1, random_state=9),
             n_jobs=-1,
             param_grid={'max_depth': range(1, 7),
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': range(90, 120, 10)},
             scoring='neg_mean_squared_error')

Обучим модель на основании лучших параметров и оценим время обучения. После этого также оценим время предсказания.

In [46]:
forest_cv.refit_time_

9.250274896621704

In [47]:
%%time
forest_cv.predict(X_train)

Wall time: 693 ms


array([3006.16517126, 9096.88577545, 1242.04076335, ..., 2224.65066609,
       2753.89560894, 3178.6263582 ])

Наконец, оценим качество полученой модели. 

In [48]:
print('RMSE cлучайного леса:', (-forest_cv.best_score_)**0.5)

RMSE cлучайного леса: 2334.7626357000854


Модель случайного леса в целом показала себя хуже метода ближайших соседей как по времени обучения, так и по качеству.

### LightGBM

Также построим модель LightGBM. Подберем наилучшие параметры для минимизации RMSE.

In [51]:
gbm = LGBMRegressor(random_state=9, n_jobs=-1)
params_gbm = {'max_depth': range (1, 7, 1), 'n_estimators': range(90, 120, 10)}
gbm_cv = RandomizedSearchCV(gbm, params_gbm, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
gbm_cv.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LGBMRegressor(random_state=9), n_jobs=-1,
                   param_distributions={'max_depth': range(1, 7),
                                        'n_estimators': range(90, 120, 10)},
                   scoring='neg_mean_squared_error')

Обучим модель с наилучшими параметрами и оценим скорость обучения. 

In [52]:
gbm_cv.refit_time_

1.9956283569335938

Также оценим время предсказания и выведем качество модели.

In [53]:
%%time
gbm_cv.predict(X_train)

Wall time: 971 ms


array([3338.27088927, 9139.12028931,  976.5754243 , ..., 1545.11623362,
       2195.38021404, 3236.85092905])

In [54]:
print('RMSE LightGBM регрессии:', (-gbm_cv.best_score_)**0.5)

RMSE LightGBM регрессии: 1927.8220185251942


На текущий момент LightGBM показала лучшее время как обучения, так и предсказания, однако по качеству она все-таки немного проигрывает модели, полученной методом ближайших соседей.

### CatBoost

Перейдем к заключительной модели CatBoost. Как и прежде начнем с подбора наиболее оптимальных параметров. 

In [55]:
cat = CatBoostRegressor(random_state=9, verbose=False)
params_cat = {'n_estimators': range(90, 120, 10),
               'depth': range (1, 7, 1), 'grow_policy': ['Depthwise']}
cat_cv = RandomizedSearchCV(cat, params_cat, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
cat_cv.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x000001E858565AC0>,
                   n_jobs=-1,
                   param_distributions={'depth': range(1, 7),
                                        'grow_policy': ['Depthwise'],
                                        'n_estimators': range(90, 120, 10)},
                   scoring='neg_mean_squared_error')

Обучим модель с использованием лучших значений для выбранных параметров и оценим время обучения. После этого также оценим время предсказаний. 

In [56]:
cat_cv.refit_time_

11.174674987792969

In [57]:
cat_cv.predict(X_train)

array([3452.72451052, 8501.86787323,  926.89857837, ..., 1642.6657838 ,
       2070.12492789, 3842.98477219])

Наконец, выведем качество последней модели. 

In [58]:
print('RMSE CatBoost регрессии:', (-cat_cv.best_score_)**0.5)

RMSE CatBoost регрессии: 1868.112872598842


Модели CatBoost не удалось обойти LightGBM по скорости обучения и предсказания, однако она показала лучшее значение качества.

Лучшей моделью относительно RMSE оказалась модель CatBoost. В то же время лучшей относительно времени оказалась LightGBM регрессия. Поскольку заказчик не уточнял, какой из критериев является наиболее значимым (и уточнить данную информацию мы не можем), нами было принято решение предложить заказчику две модели - одну с лучшим качеством, и другую с лучшим временем.

## Проверка на тесте

Оценим качество итоговой модели на тестовой выборке. Напоминаем, что в качестве итоговой были выбраны две модели - CatBoost и LightGBM.

In [59]:
test_pred_cat = cat_cv.predict(X_test)

print('RMSE CatBoostRegressor на тестовой выборке:', mean_squared_error(y_test, test_pred_cat,  squared=False))

RMSE CatBoostRegressor на тестовой выборке: 1878.3935447564163


In [60]:
test_pred_gbm = gbm_cv.predict(X_test)

print('RMSE LightGBM на тестовой выборке:', mean_squared_error(y_test, test_pred_gbm,  squared=False))

RMSE LightGBM на тестовой выборке: 1947.0081547074824


По итогу мы получили примерно одинаковое качество от обеих моделей на тестовой выборке. Качество обеих моделей на тестовых данных не слишком увеличилось, на основании чего можно сделать вывод, что если модели и переобучились, то только чуть-чуть.

## Итоговые выводы

Нами была проведена работа по созданию модели для прогнозирования рыночной стоимости автомобиля.

Прежде всего, была проведена первичная обработка исходных данных, включающая в себя выявление и устранение наблюдений с пропусками, исследование и устранение аномальных значений, а также отбор признаков для дальнейшего моделирования. Также мы обработали часть исходных переменных для возможности их использования при дальнейшем обучении.

Мы рассмотрели пять моделей в качестве потенциальных итоговых: метод k-ближайших соседей, линейная регрессия, случайный лес, LightGBM и CatBoost. Для каждой модели мы подобрали наиболее оптимальные параметры, которые бы минимизировали RMSE - показатель от заказчика. После этого мы оценили время обучения каждой из моделей с использованием оптимальных параметров, а также оценили время на предсказания. Наконец, мы также рассчитали RMSE для каждой из моделей.

По итогу лучшей были выбраны две модели - CatBoost и LightGBM. После мы проверили качество итоговых моделей на тестовой выборке. Качество CatBoost несколько сократилось, что свидетельствует о наличии переобучения. В то же время качество LightGBM сократилось меньше.